In [ ]:
### **0. Install Required Libraries**
# !pip install numpy==1.23.5
# !pip install --upgrade gensim
# !pip install --upgrade pythainlp
# !pip install emoji
# !pip install fasttext-wheel  # สำหรับ fastText embeddings

### **1. Import Libraries**
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
import string
import emoji
import warnings
warnings.filterwarnings('ignore')

# Thai NLP - เครื่องมือประมวลผลภาษาไทย
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_negations
from pythainlp.word_vector import WordVector

# Machine Learning - เครื่องมือแมชชีนเลิร์นนิง
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.sparse import hstack, csr_matrix
from collections import Counter

# FastText - เครื่องมือสำหรับ word embedding
import fasttext
import fasttext.util

# NLTK สำหรับภาษาอังกฤษ
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as nltk_tokenize
from nltk.stem import WordNetLemmatizer

# ดาวน์โหลด NLTK data (รันครั้งแรกเท่านั้น)
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('wordnet')

## **2. Load and Explore Data**
# กำหนดการแสดงผลภาษาไทย
import matplotlib
matplotlib.rcParams['font.family'] = 'DejaVu Sans'

# โหลด dataset ภาษาไทย (รีวิวหอพัก)
df_thai = pd.read_csv("Data/dorm_reviews.csv")
# คาดว่าไฟล์ไทยมีคอลัมน์ text, rating อยู่แล้ว
if 'dormitory_id' in df_thai.columns:
    df_thai = df_thai.drop([c for c in ['dormitory_id', 'user_id'] if c in df_thai.columns], axis=1)
df_thai = df_thai.rename(columns={'Review':'text','Rating':'rating'})
df_thai = df_thai[['text','rating']].copy()
df_thai['language'] = 'thai'

# โหลด dataset ภาษาอังกฤษ (รีวิวโรงแรม)
df_english = pd.read_csv("Data/tripadvisor_hotel_reviews.csv")
# ให้เหลือคอลัมน์ text, rating
if {'Review','Rating'}.issubset(df_english.columns):
    df_english = df_english[['Review','Rating']].copy()
    df_english.columns = ['text','rating']
else:
    # เผื่อไฟล์ใช้ชื่อคอลัมน์อื่น
    possible_text = [c for c in df_english.columns if c.lower() in ['review','text','comment','content']]
    possible_rating = [c for c in df_english.columns if c.lower() in ['rating','score','stars']]
    df_english = df_english[[possible_text[0], possible_rating[0]]].copy()
    df_english.columns = ['text','rating']
df_english['language'] = 'english'

# รวมสองชุดเป็น df (แก้ NameError เดิม)
df = pd.concat([df_thai, df_english], ignore_index=True)

# ทำ rating ให้เป็น int 1..5
df['rating'] = pd.to_numeric(df['rating'], errors='coerce').clip(1,5).astype(int)

### **3. โหลด Word Embedding Models**
# โหลด Thai2Vec model สำหรับภาษาไทย
print("กำลังโหลด Thai2Vec model...")
thai2fit_model = WordVector(model_name="thai2fit_wv").get_model()

# ดาวน์โหลดและโหลด fastText model สำหรับภาษาอังกฤษ
print("กำลังโหลด fastText model...")
fasttext_model = fasttext.load_model('cc.en.300.bin')
print("โหลด FastText model สำเร็จ")

def enhanced_sentence_vectorizer(text, dim=300):
    """
    ฟังก์ชันแปลงประโยคเป็น vector โดยใช้ Thai2Vec และ fastText
    """
    if not isinstance(text, str):
        text = ""
    is_thai = any(ord(char) >= 3584 and ord(char) <= 3711 for char in text)
    words = word_tokenize(text, engine="newmm") if is_thai else nltk_tokenize(text.lower())

    vec = np.zeros(dim, dtype=np.float32)
    word_count = 0
    total_words = len(words)
    
    for i, word in enumerate(words):
        if word.strip():
            position_weight = 1.0 + (i / max(total_words, 1)) * 0.5
            word_vector = None
            
            if is_thai:
                if word in thai2fit_model:
                    word_vector = thai2fit_model[word] * position_weight
                else:
                    char_vec, char_count = np.zeros(dim, dtype=np.float32), 0
                    for char in word:
                        if char in thai2fit_model:
                            char_vec += thai2fit_model[char]
                            char_count += 1
                    if char_count > 0:
                        word_vector = (char_vec / char_count) * position_weight
            else:
                try:
                    word_vector = fasttext_model.get_word_vector(word) * position_weight
                except Exception:
                    pass
            
            if word_vector is not None:
                vec += word_vector
                word_count += 1
    
    return vec / max(word_count, 1)

### **4. ฟังก์ชันทำความสะอาดข้อความและการสกัดคุณลักษณะ**
def clean_text_thai(text):
    if not isinstance(text, str):
        return ""
    from pythainlp.corpus import thai_stopwords
    thai_stop_words = list(thai_stopwords())
    important_words = [
        "ไม่","ไม่มี","ไม่ได้","ไม่ค่อย","ไม่เคย","ห้าม","ยังไม่","ไม่ยอม",
        "ดี","ดีมาก","สะอาด","เย็น","กว้าง","ใหม่","สวย","น่าอยู่","สบาย",
        "ชอบ","ประทับใจ","สะดวก","ปลอดภัย","คุ้ม","คุ้มค่า","เงียบ","ครบ",
        "พอใจ","เร็ว","โอเค","โอเคเลย","เยี่ยม","ถูกใจ","ทำเลดี","ใกล้","ครบครัน",
        "แย่","ไม่ดี","เหม็น","ร้อน","แคบ","เก่า","สกปรก","พัง","เสียงดัง",
        "แพง","ไม่ชอบ","แออัด","รก","อันตราย","ช้า","ผิดหวัง","ห่วย","เฟล",
        "กาก","ไม่คุ้ม","ไกล","รั่ว","ทรุด","ทรุดโทรม","เสื่อม",
        "มาก","สุดๆ","เยอะ","น้อย","ที่สุด","หลาย","ทุก","เกิน","จัด","โคตร",
        "มากๆ","สุดยอด","ธรรมดา","พอใช้","บ่อย","ตลอด","เวอร์",
        "แอร์","น้ำ","ไฟ","ห้องน้ำ","เตียง","ฝักบัว","เน็ต","ไวไฟ","ไฟฟ้า",
        "ประปา","เฟอร์","ลิฟต์","ที่จอด","จอดรถ","ซักผ้า","ตู้เย็น","ทีวี",
        "จาน","ไมโครเวฟ","เตา","น้ำอุ่น","ผ้าปู","โต๊ะ","เก้าอี้","ตู้","ชั้นวาง",
        "ปลั๊ก","สัญญาณ",
        "เสียง","มด","แมลง","แมลงสาบ","หนู","ยุง","ฝุ่น","กลิ่น","เพื่อนบ้าน",
        "ข้างห้อง","ข้างนอก","ถนน","ทางเดิน","ลานจอด","ชั้นบน","บันได","กำแพง",
        "ดูแล","บริการ","ซ่อม","แก้ไข","จัดการ","พนักงาน","แม่บ้าน","รปภ",
        "เจ้าของ","นิติ","กฎ","ระเบียบ","ค่าเช่า","ค่าไฟ","ค่าน้ำ","ค่าส่วนกลาง",
        "มัดจำ","ประกัน","สัญญา","ฝากของ","รับพัสดุ","คีย์การ์ด","ล็อค","รอนาน",
        "ไม่มาดู","ไม่ซ่อม",
        "แต่","แต่ว่า","ถึงแม้","อย่างไรก็ตาม","เพราะ","เพราะว่า","เนื่องจาก",
        "คือ","ก็คือ","ส่วน","นอกจากนี้","ที่จริง","จริงๆ","ก็","แม้","ที่",
        "ตอนแรก","พอดี","แล้วก็"
    ]
    custom_stop_words = [w for w in thai_stop_words if w not in important_words]
    punct = string.punctuation.replace('!', '').replace('?', '').replace('.', '')
    text = emoji.replace_emoji(text, replace="")
    text = ''.join(ch if ch not in punct else ' ' for ch in text)
    text = re.sub(r'([ก-๙a-zA-Z])\1{2,}', r'\1\1', text)
    text = re.sub(r'\s+', " ", text).strip().lower()
    words = word_tokenize(text, engine='newmm')
    words = [w for w in words if w not in custom_stop_words]
    return ' '.join(words)

def clean_text_english(text):
    if not isinstance(text, str):
        return ""
    english_stop_words = set(stopwords.words('english'))
    important_english_words = {
        'not','no','never','nothing','nowhere','neither','nobody','none',
        'good','great','excellent','amazing','wonderful','perfect','love',
        'best','nice','clean','comfortable','convenient','recommend',
        'bad','terrible','awful','horrible','worst','hate','dirty',
        'uncomfortable','expensive','cheap','noisy','small',
        'very','really','extremely','quite','pretty','too','so','absolutely'
    }
    custom_stop_words = english_stop_words - important_english_words
    punct = string.punctuation.replace('!', '').replace('?', '').replace('.', '')
    text = emoji.replace_emoji(text, replace="")
    text = ''.join(ch if ch not in punct else ' ' for ch in text)
    text = re.sub(r'([a-zA-Z])\1{2,}', r'\1\1', text)
    text = re.sub(r'\s+', " ", text).strip().lower()
    words = nltk_tokenize(text)
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(w) for w in words if w not in custom_stop_words and w.isalpha()]
    return ' '.join(words)

def clean_text(text):
    is_thai = any(ord(c) >= 3584 and ord(c) <= 3711 for c in str(text))
    return clean_text_thai(text) if is_thai else clean_text_english(text)

def extract_features(text):
    is_thai = any(ord(c) >= 3584 and ord(c) <= 3711 for c in str(text))
    words = word_tokenize(text, engine='newmm') if is_thai else nltk_tokenize(text.lower())
    word_count = len(words)
    features = {
        'exclamation_count': text.count('!'),
        'question_count': text.count('?'),
        'sentence_count': text.count('.') + 1,
        'word_count': word_count,
        'avg_word_length': sum(len(w) for w in words) / max(word_count, 1),
        'text_length': len(text)
    }
    word_counts = Counter(words)
    repeated_words = sum(1 for cnt in word_counts.values() if cnt > 1)
    features['repeated_words_ratio'] = repeated_words / max(word_count, 1)
    if is_thai:
        negation_words = set(thai_negations())
        features['negation_count'] = sum(1 for w in words if w in negation_words)
    else:
        english_negations = {'not','no','never','nothing','nobody','none','neither','nowhere'}
        features['negation_count'] = sum(1 for w in words if w in english_negations)
    features['punctuation_ratio'] = len([c for c in text if c in string.punctuation]) / max(len(text), 1)
    features['words_per_sentence'] = word_count / max(features['sentence_count'], 1)
    english_words = sum(1 for w in words if w.isascii() and w.isalpha())
    features['english_ratio'] = english_words / max(word_count, 1)
    return features

### **5. ทำความสะอาดและเตรียมข้อมูล**
print("กำลังทำความสะอาดข้อมูล...")
df['cleaned_review'] = df['text'].astype(str).apply(clean_text)

def get_word_count(text):
    is_thai = any(ord(char) >= 3584 and ord(char) <= 3711 for char in str(text))
    return len(word_tokenize(text, engine='newmm')) if is_thai else len(nltk_tokenize(text))

df = df[df['cleaned_review'].apply(get_word_count) > 3]
df = df.drop_duplicates(subset=['cleaned_review'])

print("กำลังสกัดคุณลักษณะจากข้อความ...")
feature_columns = ['cleaned_review']
feature_names = ['exclamation_count','question_count','sentence_count','word_count',
                 'avg_word_length','repeated_words_ratio','negation_count',
                 'punctuation_ratio','text_length','words_per_sentence','english_ratio']

for f in feature_names:
    df[f] = df['cleaned_review'].apply(lambda x: extract_features(x)[f])

feature_columns.extend(feature_names)

print(f"จำนวนข้อมูลหลังทำความสะอาด: {len(df)}")
print("การกระจายของคะแนนหลังทำความสะอาด:")
print(df['rating'].value_counts().sort_index())
print("\nการกระจายตามภาษาหลังทำความสะอาด:")
print(df['language'].value_counts())

### **6. แบ่งข้อมูล**
X_train, X_test, y_train, y_test = train_test_split(
    df[feature_columns], df['rating'],
    test_size=0.2, random_state=42, stratify=df['rating'] if df['rating'].nunique()>1 else None
)
print(f"จำนวนข้อมูลฝึกฝน: {len(X_train)}")
print(f"จำนวนข้อมูลทดสอบ: {len(X_test)}")

### **7. สร้าง Feature Vectors**
def custom_tokenizer_mixed(text):
    is_thai = any(ord(char) >= 3584 and ord(char) <= 3711 for char in str(text))
    return word_tokenize(text, engine='newmm') if is_thai else nltk_tokenize(text.lower())

print("กำลังสร้าง word embeddings...")
X_train_vectors = np.array([enhanced_sentence_vectorizer(t) for t in X_train['cleaned_review']], dtype=np.float32)
X_test_vectors  = np.array([enhanced_sentence_vectorizer(t) for t in X_test['cleaned_review']], dtype=np.float32)

print("กำลังสร้าง TF-IDF features...")
tfidf_vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer_mixed,
    max_features=8000,
    ngram_range=(1,3),
    min_df=2,
    max_df=0.85,
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True
)

print("กำลังสร้าง Count vectors...")
count_vectorizer = CountVectorizer(
    tokenizer=custom_tokenizer_mixed,
    max_features=2000,
    ngram_range=(1,2),
    min_df=3,
    max_df=0.85
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['cleaned_review'])
X_test_tfidf  = tfidf_vectorizer.transform(X_test['cleaned_review'])
X_train_count = count_vectorizer.fit_transform(X_train['cleaned_review'])
X_test_count  = count_vectorizer.transform(X_test['cleaned_review'])

print("กำลัง scale vectors (embeddings)...")
scaler = StandardScaler(with_mean=False)
X_train_vectors_sparse = csr_matrix(X_train_vectors)
X_test_vectors_sparse  = csr_matrix(X_test_vectors)
X_train_vectors_scaled = scaler.fit_transform(X_train_vectors_sparse)
X_test_vectors_scaled  = scaler.transform(X_test_vectors_sparse)

numerical_features = [c for c in feature_columns if c != 'cleaned_review']
X_train_additional = X_train[numerical_features].values
X_test_additional  = X_test[numerical_features].values

features_scaler = StandardScaler()
X_train_additional_scaled = features_scaler.fit_transform(X_train_additional)
X_test_additional_scaled  = features_scaler.transform(X_test_additional)

print(f"TF-IDF features: {X_train_tfidf.shape[1]}")
print(f"Count features: {X_train_count.shape[1]}")
print(f"Word embedding features: {X_train_vectors.shape[1]}")
print(f"Additional features: {X_train_additional.shape[1]}")

### **8. รวม Features ทั้งหมด**
print("กำลังรวม features ทั้งหมด...")
X_train_combined = hstack([
    X_train_tfidf,
    X_train_count,
    X_train_vectors_scaled,
    csr_matrix(X_train_additional_scaled)
], format='csr')

X_test_combined = hstack([
    X_test_tfidf,
    X_test_count,
    X_test_vectors_scaled,
    csr_matrix(X_test_additional_scaled)
], format='csr')

print(f"ขนาดของ features รวม - train: {X_train_combined.shape}, test: {X_test_combined.shape}")

# สำรองแบบ dense สำหรับโมเดลที่ไม่รองรับ sparse
print("แปลง dense สำหรับบางโมเดล...")
X_train_combined_dense = X_train_combined.toarray()
X_test_combined_dense  = X_test_combined.toarray()

# สำหรับ Naive Bayes ต้องไม่เป็นลบ
X_train_nb = np.abs(X_train_combined_dense)
X_test_nb  = np.abs(X_test_combined_dense)

### **9. กำหนด Models**
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000, C=0.2,
                                              class_weight='balanced', solver='saga',
                                              multi_class='multinomial'),
    'Random Forest': RandomForestClassifier(random_state=42, n_jobs=-1,
                                            n_estimators=200, max_depth=15,
                                            min_samples_split=5, min_samples_leaf=2,
                                            class_weight='balanced'),
    'Linear SVM': LinearSVC(random_state=42, max_iter=2000, C=0.5,
                            class_weight='balanced', dual=False),
    'Multinomial Naive Bayes': MultinomialNB(alpha=1.0, fit_prior=True)
}

### **10. ฝึกฝนและประเมินผล Models**
results = {}
best_models = {}
training_times = {}

print("\n" + "="*50)
print("กำลังฝึกฝนและประเมินผล MODELS")
print("="*50)

for model_name, model in models.items():
    print(f"\n🔧 กำลังฝึกฝน {model_name}...")
    start_time = time.time()
    
    # เลือกชุดฟีเจอร์ที่เหมาะสม
    if model_name == 'Multinomial Naive Bayes':
        Xtr, Xte = X_train_nb, X_test_nb
    elif model_name == 'Random Forest':
        # Tree-based ไม่รับ sparse -> ใช้ dense
        Xtr, Xte = X_train_combined_dense, X_test_combined_dense
    else:
        Xtr, Xte = X_train_combined, X_test_combined
    
    model.fit(Xtr, y_train)
    best_models[model_name] = model
    training_time = time.time() - start_time
    training_times[model_name] = training_time
    
    y_pred = model.predict(Xte)
    accuracy = accuracy_score(y_test, y_pred)

    # cross_val_score: ใช้ฟีเจอร์เดียวกับตอน train
    try:
        cv_scores = cross_val_score(model, Xtr, y_train, cv=5, scoring='accuracy')
    except Exception:
        # เผื่อบางโมเดลไม่เหมาะกับ CV ในสภาพนี้
        cv_scores = np.array([accuracy])
    
    results[model_name] = {
        'accuracy': accuracy,
        'cv_mean': float(np.mean(cv_scores)),
        'cv_std': float(np.std(cv_scores)),
        'predictions': y_pred,
        'training_time': training_time
    }
    
    print(f"✅ {model_name} เสร็จสิ้น!")
    print(f"   ความแม่นยำการทดสอบ: {accuracy:.4f}")
    print(f"   ความแม่นยำ CV: {np.mean(cv_scores):.4f} (±{np.std(cv_scores):.4f})")
    print(f"   เวลาฝึกฝน: {training_time:.2f} วินาที")

### **11. เปรียบเทียบผลลัพธ์และการแสดงผล**
results_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Test Accuracy': [results[m]['accuracy'] for m in results.keys()],
    'CV Mean': [results[m]['cv_mean'] for m in results.keys()],
    'CV Std': [results[m]['cv_std'] for m in results.keys()],
    'Training Time (s)': [results[m]['training_time'] for m in results.keys()]
})

print("\n" + "="*60)
print("ผลการเปรียบเทียบ MODELS")
print("="*60)
print(results_df.round(4))

best_model_name = results_df.loc[results_df['Test Accuracy'].idxmax(), 'Model']
print(f"\n🏆 โมเดลที่มีประสิทธิภาพดีที่สุด: {best_model_name}")
print(f"   ความแม่นยำการทดสอบ: {results[best_model_name]['accuracy']:.4f}")

### **12. การแสดงผลแบบกราฟ**
plt.rcParams['font.family'] = ['DejaVu Sans']
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0, 0].bar(results_df['Model'], results_df['Test Accuracy'])
axes[0, 0].set_title('การเปรียบเทียบความแม่นยำการทดสอบ', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('ความแม่นยำ'); axes[0, 0].tick_params(axis='x', rotation=45); axes[0, 0].grid(True, alpha=0.3)
for i, v in enumerate(results_df['Test Accuracy']):
    axes[0, 0].text(i, v + 0.005, f'{v:.3f}', ha='center', fontweight='bold')

axes[0, 1].errorbar(range(len(results_df)), results_df['CV Mean'],
                    yerr=results_df['CV Std'], fmt='o-', capsize=5, linewidth=2)
axes[0, 1].set_title('ความแม่นยำ Cross-Validation (เฉลี่ย ± ส่วนเบี่ยงเบนมาตรฐาน)', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('ความแม่นยำ CV'); axes[0, 1].set_xticks(range(len(results_df)))
axes[0, 1].set_xticklabels(results_df['Model'], rotation=45); axes[0, 1].grid(True, alpha=0.3)

bars = axes[1, 0].bar(results_df['Model'], results_df['Training Time (s)'])
axes[1, 0].set_title('การเปรียบเทียบเวลาฝึกฝน', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('เวลา (วินาที)'); axes[1, 0].tick_params(axis='x', rotation=45); axes[1, 0].grid(True, alpha=0.3)
for bar, time_val in zip(bars, results_df['Training Time (s)']):
    axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, f'{time_val:.1f}s', ha='center', fontweight='bold')

# Radar
categories = ['ความแม่นยำทดสอบ', 'ความแม่นยำ CV', 'ความเร็ว (กลับหัว)']
fig2, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))
normalized_accuracy = results_df['Test Accuracy'] / max(1e-9, results_df['Test Accuracy'].max())
normalized_cv = results_df['CV Mean'] / max(1e-9, results_df['CV Mean'].max())
normalized_speed = (1 / (results_df['Training Time (s)'] + 0.1))
normalized_speed = normalized_speed / max(1e-9, normalized_speed.max())
angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist(); angles += angles[:1]
base_colors = ['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9']
for i, model in enumerate(results_df['Model']):
    values = [normalized_accuracy.iloc[i], normalized_cv.iloc[i], normalized_speed.iloc[i]]; values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=model, color=base_colors[i % len(base_colors)])
    ax.fill(angles, values, alpha=0.25, color=base_colors[i % len(base_colors)])
ax.set_xticks(angles[:-1]); ax.set_xticklabels(categories); ax.set_ylim(0, 1)
ax.set_title('การเปรียบเทียบประสิทธิภาพโมเดล (ปรับมาตรฐาน)', fontweight='bold', size=14)
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0)); ax.grid(True)
axes[1, 1].remove()
plt.tight_layout(); plt.show()

### **13. รายงานการจำแนกประเภทโดยละเอียด**
print("\n" + "="*60)
print("รายงานการจำแนกประเภทโดยละเอียด")
print("="*60)

# ทำให้ปลอดภัยหากบางคลาสไม่มีใน y_test
unique_labels = sorted(df['rating'].unique().tolist())
target_names = [f'คะแนน {i}' for i in unique_labels]

for model_name in results.keys():
    print(f"\n📊 {model_name}")
    print("-" * 40)
    model = models[model_name]
    params_show = ['C','class_weight','solver','multi_class','n_estimators',
                   'max_depth','min_samples_split','min_samples_leaf','dual',
                   'alpha','fit_prior']
    param_desc = {
        'C':'ค่า regularization parameter','class_weight':'การปรับน้ำหนักคลาส',
        'solver':'อัลกอริทึมการหาค่าที่เหมาะสม','multi_class':'วิธีการจัดการหลายคลาส',
        'n_estimators':'จำนวน decision trees','max_depth':'ความลึกสูงสุดของต้นไม้',
        'min_samples_split':'จำนวนตัวอย่างขั้นต่ำในการแยก node',
        'min_samples_leaf':'จำนวนตัวอย่างขั้นต่ำใน leaf node',
        'dual':'การใช้ dual formulation','alpha':'ค่า smoothing parameter',
        'fit_prior':'การใช้ prior probability'
    }
    for p, v in model.get_params().items():
        if p in params_show:
            print(f"  {p}: {v} # {param_desc.get(p,'')}")
    print(f"\nรายงานการจำแนกประเภท:")
    # ใช้ labels และ target_names ให้สอดคล้องกัน
    print(classification_report(y_test, results[model_name]['predictions'],
                                labels=unique_labels, target_names=target_names, zero_division=0))

### **14. Confusion Matrices**
rows = int(np.ceil(len(results)/2))
cols = 2 if len(results) > 1 else 1
fig, axes = plt.subplots(rows, cols, figsize=(15, 6*rows))
axes = np.array(axes).reshape(-1)
for i, (model_name, result) in enumerate(results.items()):
    cm = confusion_matrix(y_test, result['predictions'], labels=unique_labels)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=[str(l) for l in unique_labels],
                yticklabels=[str(l) for l in unique_labels],
                ax=axes[i])
    axes[i].set_xlabel("คะแนนที่ทำนาย"); axes[i].set_ylabel("คะแนนจริง")
    axes[i].set_title(f"{model_name}\nความแม่นยำ: {result['accuracy']:.3f}")
plt.tight_layout(); plt.savefig("confusion_matrices_comparison.png", dpi=300, bbox_inches='tight'); plt.show()

### **15. ฟังก์ชันทำนายผลที่ปรับปรุง**
def predict_review_enhanced(text, model_name='best'):
    if model_name == 'best':
        model_name = best_model_name
    model = best_models[model_name]
    cleaned_text = clean_text(text)
    features = extract_features(cleaned_text)

    review_vector = enhanced_sentence_vectorizer(cleaned_text).reshape(1, -1)
    review_vector_sparse = csr_matrix(review_vector)
    review_vector_scaled = scaler.transform(review_vector_sparse)

    review_tfidf = tfidf_vectorizer.transform([cleaned_text])
    review_count = count_vectorizer.transform([cleaned_text])

    additional_features = np.array([[
        features['exclamation_count'], features['question_count'],
        features['sentence_count'], features['word_count'],
        features['avg_word_length'], features['repeated_words_ratio'],
        features['negation_count'], features['punctuation_ratio'],
        features['text_length'], features['words_per_sentence'],
        features['english_ratio']
    ]], dtype=np.float32)
    additional_features_scaled = features_scaler.transform(additional_features)

    if model_name in ['Multinomial Naive Bayes','Random Forest']:
        # ใช้เวอร์ชัน dense สำหรับโมเดลที่ต้องการ non-negative/dense
        review_combined = hstack([
            review_tfidf, review_count, review_vector_scaled, csr_matrix(additional_features_scaled)
        ], format='csr').toarray()
        if model_name == 'Multinomial Naive Bayes':
            review_combined = np.abs(review_combined)
    else:
        review_combined = hstack([
            review_tfidf, review_count, review_vector_scaled, csr_matrix(additional_features_scaled)
        ], format='csr')

    pred = model.predict(review_combined)[0]
    if hasattr(model, 'predict_proba'):
        probs = model.predict_proba(review_combined)[0]
    else:
        # LinearSVC
        scores = model.decision_function(review_combined)[0]
        # ปรับ softmax แบบง่าย
        exps = np.exp(scores - np.max(scores))
        probs = exps / np.sum(exps)
    return int(pred), probs

### **16. ทดสอบกับรีวิวจริง**
def print_prediction_results_enhanced(text, actual_rating, model_name='best'):
    predicted_class, confidences = predict_review_enhanced(text, model_name)
    is_correct = int(predicted_class) == int(actual_rating)
    print(f"\nรีวิว: {text[:100]}...")
    print(f"โมเดล: {model_name}")
    print(f"คะแนนจริง: {actual_rating}/5")
    print(f"คะแนนที่ทำนาย: {predicted_class}/5")
    print(f"ผลลัพธ์: {'✅ ถูกต้อง' if is_correct else '❌ ผิดพลาด'}")
    print("ความมั่นใจ:")
    # ทำ mapping ตามลำดับคลาสที่โมเดลใช้ (สมมติเรียง 1..5)
    for rating_idx, confidence in enumerate(confidences, 1):
        print(f"  ⭐ {rating_idx}: {confidence * 100:.1f}%")
    return predicted_class, int(actual_rating)

test_reviews = [
    ["ไม่แนะนำเลยค่ะ หอนี้ หลอกเอาเงินชัดๆ ในรูปสวยมาก แต่พอเข้าไปอยู่จริงสภาพห้องทรุดโทรมมาก ตู้เสื้อผ้าพังตั้งแต่วันแรกที่ย้ายเข้า เตียงก็เก่ามากนอนแล้วปวดหลัง ฝักบัวน้ำก็ไหลแค่ซิกๆ ไม่เคยมาซ่อมให้สักที ขอย้ายออกก็ไม่คืนเงินมัดจำ เสียความรู้สึกมากค่ะ", 1],
    ["หอพักราคาก็โอเคนะ ไม่แพงมาก แต่มีข้อเสียเยอะไปหน่อย ห้องเล็กเกินไป แอร์เสียงดังรบกวนเวลานอน ประตูห้องน้ำปิดไม่สนิท แล้วก็มีมดเยอะมาก ข้อดีคือใกล้ตลาด เดินไปซื้อของกินได้สะดวก แต่ภาพรวมยังไม่คุ้มค่าเท่าไหร่ ถ้ามีทางเลือกอื่นก็น่าจะดีกว่านะ", 2],
    ["ชอบหอนี้มากค่ะ ห้องกว้างสะอาด เฟอร์นิเจอร์ครบครัน แอร์เย็นฉ่ำ มีโต๊ะเครื่องแป้งด้วย สะดวกมาก อินเทอร์เน็ตเร็ว เล่นเกมสบาย เจ้าของหอใจดี มีอะไรแจ้งปุ๊บมาดูปั๊บ ข้อเสียเล็กๆคือค่าไฟค่อนข้างแพง แล้วก็ซักผ้าต้องลงไปชั้นล่าง อยากให้มีเครื่องซักผ้าทุกชั้น แต่โดยรวมพอใจมากค่ะ แนะนำเลย", 4],
    ["Terrible hotel experience! The room was dirty, smelly, and completely different from the photos. Staff was rude and unhelpful. AC didn't work, hot water was cold, and WiFi was extremely slow. The bed was uncomfortable and the bathroom was disgusting. Would never stay here again. Complete waste of money!", 1],
    ["The hotel is okay for the price. Room was decent size but could be cleaner. Staff was friendly but service was slow. Location is good, close to attractions. Some facilities need maintenance. WiFi worked well. Overall, it's an average hotel - nothing special but acceptable for a short stay.", 3],
    ["Amazing hotel! Absolutely loved our stay here. The room was spacious, clean, and beautifully decorated. Staff was incredibly friendly and helpful. The location is perfect - walking distance to everything. Breakfast was delicious with great variety. Pool and gym facilities were excellent. Highly recommend this place!", 5],
    ["This hotel is really good! ห้องสะอาดมาก แอร์เย็น wifi super fast แต่ราคาแพงไปหน่อย but overall worth it นะ staff friendly มาก highly recommended! 👍", 4],
    ["หอนี้ดีที่สุดในย่านนี้แล้วว อยู่มา 3 ปีไม่เคยมีปัญหาเลย ห้องกว้าง สะอาด ตกแต่งสวย มีเฟอร์ครบ เหมือนอยู่คอนโด เน็ตไวมาก 100 Mbps เล่นเกมไม่มีสะดุด! ระบบรักษาความปลอดภัยแน่นมาก มีกล้องวงจรปิด คีย์การ์ดทุกชั้น และมี รปภ. 24 ชม. ทีเด็ดสุดคือมีฟิตเนสและสระว่ายน้ำให้ใช้ฟรี คุ้มมากกกก แนะนำสุดๆ ถ้าได้ห้องก็จองเลยอย่ารอ!", 5]
]

print("\n" + "="*60)
print("การทดสอบกับรีวิวจริง")
print("="*60)
print(f"🏆 ทดสอบกับโมเดลที่ดีที่สุด: {best_model_name}")
test_results = []
for review, actual_rating in test_reviews:
    predicted_rating, actual = print_prediction_results_enhanced(review, actual_rating, best_model_name)
    test_results.append((predicted_rating, actual))

correct_predictions = sum(1 for pred, actual in test_results if pred == actual)
test_accuracy = correct_predictions / len(test_results)
print(f"\n📈 ผลสรุปการทดสอบรีวิว (โมเดลที่ดีที่สุด: {best_model_name}):")
print(f"ทำนายถูกต้อง: {correct_predictions} รีวิว")
print(f"ทำนายผิดพลาด: {len(test_results) - correct_predictions} รีวิว")
print(f"ความแม่นยำรวม: {test_accuracy:.2f} ({correct_predictions}/{len(test_results)})")

### **17. เปรียบเทียบโมเดลกับรีวิวทดสอบ**
print("\n" + "="*60)
print("เปรียบเทียบทุกโมเดลกับรีวิวทดสอบ")
print("="*60)
model_test_results = {}
for model_name in results.keys():
    print(f"\n🔍 ทดสอบ {model_name}:")
    model_results = []
    for idx, (review, actual_rating) in enumerate(test_reviews[:5]):
        predicted_rating, _ = predict_review_enhanced(review, model_name)
        model_results.append(predicted_rating == actual_rating)
        print(f"  รีวิวที่ {idx+1}: {'✅' if model_results[-1] else '❌'}")
    accuracy_small = sum(model_results) / len(model_results)
    model_test_results[model_name] = accuracy_small
    print(f"  ความแม่นยำในตัวอย่าง: {accuracy_small:.2f}")

### **18. การวิเคราะห์ Feature Importance (สำหรับ tree-based models)**
if 'Random Forest' in best_models:
    print(f"\n🌳 การวิเคราะห์ความสำคัญของ Features (Random Forest):")
    rf_model = best_models['Random Forest']
    tfidf_features = [f"tfidf_{i}" for i in range(X_train_tfidf.shape[1])]
    count_features = [f"count_{i}" for i in range(X_train_count.shape[1])]
    embedding_features = [f"embed_{i}" for i in range(X_train_vectors.shape[1])]
    all_feature_names = tfidf_features + count_features + embedding_features + feature_names
    feature_importance = rf_model.feature_importances_
    top_k = min(20, feature_importance.shape[0])
    top_indices = np.argsort(feature_importance)[-top_k:]
    plt.figure(figsize=(10, 8))
    plt.barh(range(top_k), feature_importance[top_indices])
    plt.yticks(range(top_k), [all_feature_names[i] for i in top_indices])
    plt.xlabel('ความสำคัญของ Feature'); plt.title(f'{top_k} Features ที่สำคัญที่สุด (Random Forest)')
    plt.tight_layout(); plt.savefig("feature_importance.png", dpi=300, bbox_inches='tight'); plt.show()

### **19. สรุปประสิทธิภาพ**
print("\n" + "="*60)
print("สรุปประสิทธิภาพสุดท้าย")
print("="*60)
summary_df = pd.DataFrame({
    'Model': results_df['Model'],
    'Test Accuracy': results_df['Test Accuracy'],
    'CV Accuracy': results_df['CV Mean'],
    'Std Dev': results_df['CV Std'],
    'Training Time': results_df['Training Time (s)'],
    'Rank': results_df['Test Accuracy'].rank(ascending=False).astype(int)
}).sort_values('Test Accuracy', ascending=False)
print(summary_df.round(4))

print(f"\n🎯 ข้อมูลเชิงลึกสำคัญ:")
print(f"• โมเดลที่ดีที่สุด: {best_model_name} (ความแม่นยำ: {results[best_model_name]['accuracy']:.4f})")
print(f"• การใช้ FastText + Thai2Vec embeddings ช่วยจัดการภาษาผสมได้ดีขึ้น")
print(f"• โมเดลสามารถจัดการคำภาษาอังกฤษในรีวิวไทยได้แล้ว")
print(f"• Features ทั้งหมด: {X_train_combined.shape[1]:,}")
print(f"• รองรับทั้งรีวิวภาษาไทยและอังกฤษ")


[nltk_data] Downloading package punkt to C:\Users\Phutawan
[nltk_data]     Chonsakorn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Phutawan
[nltk_data]     Chonsakorn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Phutawan
[nltk_data]     Chonsakorn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


กำลังโหลด Thai2Vec model...
กำลังโหลด fastText model...
โหลด FastText model สำเร็จ
กำลังทำความสะอาดข้อมูล...
กำลังสกัดคุณลักษณะจากข้อความ...
จำนวนข้อมูลหลังทำความสะอาด: 22493
การกระจายของคะแนนหลังทำความสะอาด:
rating
1    1821
2    2191
3    2584
4    6448
5    9449
Name: count, dtype: int64

การกระจายตามภาษาหลังทำความสะอาด:
language
english    20491
thai        2002
Name: count, dtype: int64
จำนวนข้อมูลฝึกฝน: 17994
จำนวนข้อมูลทดสอบ: 4499
กำลังสร้าง word embeddings...
กำลังสร้าง TF-IDF features...
กำลังสร้าง Count vectors...
กำลัง scale vectors (embeddings)...
TF-IDF features: 8000
Count features: 2000
Word embedding features: 300
Additional features: 11
กำลังรวม features ทั้งหมด...
ขนาดของ features รวม - train: (17994, 10311), test: (4499, 10311)
แปลง dense สำหรับบางโมเดล...

กำลังฝึกฝนและประเมินผล MODELS

🔧 กำลังฝึกฝน Logistic Regression...
✅ Logistic Regression เสร็จสิ้น!
   ความแม่นยำการทดสอบ: 0.6026
   ความแม่นยำ CV: 0.5979 (±0.0126)
   เวลาฝึกฝน: 183.43 วินาที

🔧 กำลังฝึกฝน Random